<img src="https://openfisca.org/img/logo-openfisca.svg" width="198" align="right">
<h1 id="tocheading">Getting Started</h1>

This notebook illustrates the [key concepts](https://openfisca.org/doc/key-concepts/index.html) of OpenFisca that you will find in the documentation.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


## Calculate a variable

Do some imports.

In [100]:
from pprint import pprint  # data pretty printer

from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem

Initialize the French [Tax and Benefit system](https://openfisca.org/doc/key-concepts/tax_and_benefit_system.html): it launches the entire legislation.

In [101]:
tax_benefit_system = FranceTaxBenefitSystem()

Initialize a test case: first step to give a persons situations for which you want to calculate taxes or benefits.  
Here we start with the persons: we have one parent and two children.

In [102]:
TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2015-01': 30},
            'salaire_de_base': {'2015': 20000}
        },
        'enfant1': {
            'age': {'2015-01': 12}
        },
        'enfant2': {
            'age': {'2015-01': 18}
        }
    }
}

Define the role of those persons as members of `foyers_fiscaux` (tax households), `menages` (social households) and `familles` (families) groups.  
These groups come with the tax and benefit system.

In [103]:
TEST_CASE['foyers_fiscaux'] = {
    'foyer_fiscal1': {
        'declarants': ['parent1'],
        'personnes_a_charge': ['enfant1', 'enfant2']
    }
}

TEST_CASE['menages'] = {
    'menage1': {
        'personne_de_reference': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

TEST_CASE['familles'] = {
    'famille1': {
        'parents': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

# display full test case
pprint(TEST_CASE)

{'familles': {'famille1': {'enfants': ['enfant1', 'enfant2'],
                           'parents': ['parent1']}},
 'foyers_fiscaux': {'foyer_fiscal1': {'declarants': ['parent1'],
                                      'personnes_a_charge': ['enfant1',
                                                             'enfant2']}},
 'individus': {'enfant1': {'age': {'2015-01': 12}},
               'enfant2': {'age': {'2015-01': 18}},
               'parent1': {'age': {'2015-01': 30},
                           'salaire_de_base': {'2015': 20000}}},
 'menages': {'menage1': {'enfants': ['enfant1', 'enfant2'],
                         'personne_de_reference': ['parent1']}}}


And use this test case to create a [simulation](https://openfisca.org/doc/key-concepts/simulation.html#simulation-the-framework-of-computation): the OpenFisca frame for calculating taxes or benefits.

In [104]:
simulation_builder = SimulationBuilder()
simulation = simulation_builder.build_from_entities(tax_benefit_system, TEST_CASE)

Calculate a variable: `af` (family allowance) for example.

In [105]:
simulation.calculate('af', '2015-01')

array([129.99], dtype=float32)

## Test the impact of a reform

A reform represents a modified version of the real Tax and Benefit legislation* 

Reforms are gather in the module [`Reform`](https://github.com/openfisca/openfisca-france/tree/master/openfisca_france/reforms) of OpenFisca.

You have then to import the module extension corresponding to the reform of your concern.

Here we choose the fiscal reform of Landais, Piketty and Saez on the income tax (http://www.revolution-fiscale.fr/la-reforme-proposee). 

It is already implemented in OpenFisca under the name `landais_piketty_saez` .

In [106]:
from openfisca_france.reforms import landais_piketty_saez

Create a modified version of the tax and benefit system, affected by the changes introduced by the reform

In [107]:
reform = landais_piketty_saez.landais_piketty_saez(tax_benefit_system)

Create the scenario you want to test.

In [108]:
def init_profile(scenario):
    scenario.init_single_entity(
        period = '2013',
        parent1 = dict(
            age = 40,
            salaire_de_base = 50000,
            ),
        )
    return scenario

With the reform

In [109]:
# Indicate that you want to perfom the reform on this scenario
reform_scenario = init_profile(reform.new_scenario())

#Simulate the reform
reform_simulation = reform_scenario.new_simulation() 

# Choose the variable you want to calcul : here the disposable income, "revenu_disponible"
reform_simulation.calculate('revenu_disponible', '2013')

AttributeError: 'ScenarioAdapter' object has no attribute 'init_single_entity'

Without the reform: the baseline (counterfactual)

In [ ]:
# Indicate that you want to perfom the standard system on this scenario
baseline_scenario = init_profile(tax_benefit_system.new_scenario())

# Simulate the standard scenario
baseline_simulation = baseline_scenario.new_simulation()

# Choose the variable you want to calcul
baseline_simulation.calculate('revenu_disponible', '2013')

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')